# RA2CE feature: Isolated locations
**NOTE: TEXT WILL BE UPDATED**

This notebook contains an example of the **isolated locations analysis** of the RA2CE model and can be used for any location in the world where OSM data is available. This requires the right folder setup, and the correct settings in the configuration files (network.ini and analyses.ini).

First of all, we will import the packages we need to execute the notebook:

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from ra2ce.ra2ce_handler import Ra2ceHandler #import the ra2cehandler to run ra2ce analyses

c:\Users\groen_fe\.conda\envs\ra2ce_env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
c:\Users\groen_fe\.conda\envs\ra2ce_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Then, we have to set some basic variables. Keep in mind that you will have to specify the path to your local machine yourself.
First, we will set the path to the ra2ce folder and we will initialize the names for the network and analysis ini files.

Afterwards, let's find the examples folder on your machine.

In [2]:
root_dir = Path(r'c:\Users\groen_fe\OneDrive - Stichting Deltares\2_Products\ra2ce\database\isolated_locations') # Set the path to your examples folder

# Isolated locations analysis
EXPLANATION

**Network.ini content**
<br>
<br>
> [project]<br>
name = isolated_locations<br>
<br>
[network]<br>
directed = False<br>
source = OSM download<br>
primary_file = None<br>
diversion_file = None<br>
file_id = None<br>
polygon = kampala.geojson<br>
network_type = drive<br>
road_types = motorway,motorway_link,trunk,trunk_link,primary,primary_link,secondary,secondary_link,tertiary,tertiary_link<br>
save_shp = True<br>
<br>
[origins_destinations]<br>
origins = None<br>
destinations = None<br>
origins_names = A<br>
destinations_names = B<br>
id_name_origin_destination = OBJECTID<br>
origin_count = POPULATION<br>                         
origin_out_fraction = 1<br>
category = category<br>
<br>
[hazard]<br>
**hazard_map = DFO_3170_From_20070815_to_20071031_flooded.tif**<br>
hazard_id = None<br>
hazard_field_name = waterdepth<br>
**aggregate_wl = max**<br>
**hazard_crs = 4326**<br>
<br>
*[cleanup]<br>
snapping_threshold = None<br>
segmentation_length = None<br>
merge_lines = False<br>
merge_on_id = False<br>
cut_at_intersections = False<br>*


In [3]:
_network_ini_name = "network.ini" #set the name for the network.ini
_analyses_ini_name = "analyses.ini" #set the name for the analysis.ini

network_ini = root_dir / _network_ini_name
assert network_ini.is_file()

analyses_ini = root_dir / _analyses_ini_name
assert analyses_ini.is_file()

In [4]:
handler = Ra2ceHandler(network=network_ini, analysis=analyses_ini)
handler.configure()
# handler.run_analysis()

2023-08-25 04:51:37 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-08-25 04:51:37 PM - [network_config_wrapper.py:106] - root - INFO - Existing graph/network found: c:\Users\groen_fe\OneDrive - Stichting Deltares\2_Products\ra2ce\database\isolated_locations\static\output_graph\base_graph.p.
2023-08-25 04:51:37 PM - [network_config_wrapper.py:106] - root - INFO - Existing graph/network found: c:\Users\groen_fe\OneDrive - Stichting Deltares\2_Products\ra2ce\database\isolated_locations\static\output_graph\base_network.feather.
2023-08-25 04:51:37 PM - [network_config_wrapper.py:106] - root - INFO - Existing graph/network found: c:\Users\groen_fe\OneDrive - Stichting Deltares\2_Products\ra2ce\database\isolated_locations\static\output_graph\base_graph_hazard.p.
2023-08-25 04:51:37 PM - [networks.py:183] - root - INFO - Apparently, you already did create a network with ra2ce earlier. Ra2ce will use this: c:\Users\groen_fe\OneDrive - Stichting Deltares\2_Products\ra

DriverError: c:\Users\groen_fe\OneDrive - Stichting Deltares\2_Products\ra2ce\database\isolated_locations\static\network: Permission denied